In [1]:
from tmilib import *

In [2]:
from reconstruct_focus_times_common import *
from reconstruct_focus_times import ReconstructFocusTimesBaseline
from session_tracker import get_focused_tab

In [3]:
'''
for user in list_users():
  ordered_visits = get_history_ordered_visits_for_user(user)
  if len(ordered_visits) == 0:
    continue
  last_visit = ordered_visits[-1]
  last_visit_time = float(last_visit['visitTime']) / 1000.0
  print user, datetime.datetime.fromtimestamp(last_visit_time)
'''

"\nfor user in list_users():\n  ordered_visits = get_history_ordered_visits_for_user(user)\n  if len(ordered_visits) == 0:\n    continue\n  last_visit = ordered_visits[-1]\n  last_visit_time = float(last_visit['visitTime']) / 1000.0\n  print user, datetime.datetime.fromtimestamp(last_visit_time)\n"

In [4]:
'''
for user in list_users():
  tab_focus_times = get_tab_focus_times_for_user(user)
  if len(tab_focus_times) == 0:
    continue
  last_visit = tab_focus_times[-1]
  last_visit_time = float(last_visit['end']) / 1000.0
  print user, datetime.datetime.fromtimestamp(last_visit_time)
'''

"\nfor user in list_users():\n  tab_focus_times = get_tab_focus_times_for_user(user)\n  if len(tab_focus_times) == 0:\n    continue\n  last_visit = tab_focus_times[-1]\n  last_visit_time = float(last_visit['end']) / 1000.0\n  print user, datetime.datetime.fromtimestamp(last_visit_time)\n"

In [5]:
def ignore_all_before_start_or_after_end(visit_lengths, start_time, end_time):
  output = []
  for x in visit_lengths:
    if x['end'] < start_time:
      continue
    if x['start'] > end_time:
      continue
    #if x['start'] < start_time: # - 1000: # 1 second before or after
    #  continue
    #if x['end'] > end_time: # + 1000:
    #  continue
    output.append(x)
  return output

In [6]:
def get_evaluated_reference_and_reconstructed_focus_times_for_user(user):
  #tab_focus_times = get_tab_focus_times_for_user(user)
  #tab_focus_times = get_tab_focus_times_only_tab_updated_for_user(user)
  tab_focus_times = get_tab_focus_times_only_tab_updated_urlchanged_for_user(user)
  reconstructed_tab_focus_times = list(get_reconstruct_focus_times_baseline_for_user(user))
  if len(tab_focus_times) == 0 or len(reconstructed_tab_focus_times) == 0:
    return [],[]
  ref_start_time = max(get_earliest_start_time(tab_focus_times), get_earliest_start_time(reconstructed_tab_focus_times))
  ref_end_time = min(get_last_end_time(tab_focus_times), get_last_end_time(reconstructed_tab_focus_times))
  evaluated_reconstructed_tab_focus_times = ignore_all_before_start_or_after_end(reconstructed_tab_focus_times, ref_start_time, ref_end_time)
  evaluated_tab_focus_times = ignore_all_before_start_or_after_end(tab_focus_times, ref_start_time, ref_end_time)
  return evaluated_tab_focus_times,evaluated_reconstructed_tab_focus_times

def get_evaluation_stats_for_user(user):
  evaluated_tab_focus_times,evaluated_reconstructed_tab_focus_times = get_evaluated_reference_and_reconstructed_focus_times_for_user(user)
  if len(evaluated_reconstructed_tab_focus_times) == 0 or len(evaluated_tab_focus_times) == 0:
    return {}
  return evalutate_tab_focus_reconstruction(evaluated_tab_focus_times, evaluated_reconstructed_tab_focus_times)

'''
def get_evaluation_stats_for_user(user):
  #evaluated_tab_focus_times,evaluated_reconstructed_tab_focus_times = get_evaluated_reference_and_reconstructed_focus_times_for_user(user)
  #if len(evaluated_reconstructed_tab_focus_times) == 0 or len(evaluated_tab_focus_times) == 0:
  #  return {}
  tab_focus_times = get_tab_focus_times_for_user(user)
  reconstructed_tab_focus_times = list(get_reconstruct_focus_times_baseline_for_user(user))
  if len(tab_focus_times) == 0 or len(reconstructed_tab_focus_times) == 0:
    return {}
  ref_start_time = max(get_earliest_start_time(tab_focus_times), get_earliest_start_time(reconstructed_tab_focus_times))
  ref_end_time = min(get_last_end_time(tab_focus_times), get_last_end_time(reconstructed_tab_focus_times))
  evaluated_reconstructed_tab_focus_times = ignore_all_before_start_or_after_end(reconstructed_tab_focus_times, ref_start_time, ref_end_time)
  evaluated_tab_focus_times = ignore_all_before_start_or_after_end(tab_focus_times, ref_start_time, ref_end_time)
  result_reference = second_to_active_url(evaluated_tab_focus_times, ref_start_time, ref_end_time)
  result_reconstructed = second_to_active_url(evaluated_reconstructed_tab_focus_times, ref_start_time, ref_end_time)
  return evaluation_stats_for_reconstruction(result_reference, result_reconstructed)
  #print tab_focus_times[0].keys()
  #print list(reconstructed_tab_focus_times)[0].keys()
'''

'\ndef get_evaluation_stats_for_user(user):\n  #evaluated_tab_focus_times,evaluated_reconstructed_tab_focus_times = get_evaluated_reference_and_reconstructed_focus_times_for_user(user)\n  #if len(evaluated_reconstructed_tab_focus_times) == 0 or len(evaluated_tab_focus_times) == 0:\n  #  return {}\n  tab_focus_times = get_tab_focus_times_for_user(user)\n  reconstructed_tab_focus_times = list(get_reconstruct_focus_times_baseline_for_user(user))\n  if len(tab_focus_times) == 0 or len(reconstructed_tab_focus_times) == 0:\n    return {}\n  ref_start_time = max(get_earliest_start_time(tab_focus_times), get_earliest_start_time(reconstructed_tab_focus_times))\n  ref_end_time = min(get_last_end_time(tab_focus_times), get_last_end_time(reconstructed_tab_focus_times))\n  evaluated_reconstructed_tab_focus_times = ignore_all_before_start_or_after_end(reconstructed_tab_focus_times, ref_start_time, ref_end_time)\n  evaluated_tab_focus_times = ignore_all_before_start_or_after_end(tab_focus_times, ref_

In [7]:
class ClosestVisitTimeForUrl:
  def __init__(self, tab_focus_times):
    self.visits = tab_focus_times
    self.url_to_visit_times = {} # url to start of visit
    self.visit_time_to_visit = {}
    for visit in self.visits: # these visits are time-ordered
      start = visit['start']
      url = visit['url']
      if url not in self.url_to_visit_times:
        self.url_to_visit_times[url] = []
      self.url_to_visit_times[url].append(start)
      self.visit_time_to_visit[start] = visit
  def get_visit_from_time(self, target_visit_time):
    self.visit_time_to_visit.get(target_visit_time, None)
  def find_closest_visit(self, url, target_visit_time):
    if url not in self.url_to_visit_times:
      return None
    all_visits = self.url_to_visit_times[url]
    closest_visit = None
    for visit in all_visits:
      if closest_visit == None:
        closest_visit = visit
        continue
      if abs(visit - target_visit_time) < abs(closest_visit - target_visit_time):
        closest_visit = visit
    return closest_visit

In [8]:
def get_span_alignment_stats_for_user(user):
  evaluated_tab_focus_times,evaluated_reconstructed_tab_focus_times = get_evaluated_reference_and_reconstructed_focus_times_for_user(user)
  if len(evaluated_reconstructed_tab_focus_times) == 0 or len(evaluated_tab_focus_times) == 0:
    return {}
  closest_visit_times_for_reference = ClosestVisitTimeForUrl(evaluated_tab_focus_times)
  closest_visit_times_for_reconstructed = ClosestVisitTimeForUrl(evaluated_reconstructed_tab_focus_times)
  stats = Counter()
  domains_where_reconstructed_has_no_reference = Counter()
  domains_where_reference_has_no_reconstructed = Counter()
  urls_where_reconstructed_has_no_reference = Counter()
  urls_where_reference_has_no_reconstructed = Counter()
  reconstructed_transition_types_with_no_corresponding_reference = Counter()
  reconstructed_transition_types_total = Counter()
  reconstructed_visits_labeled = []
  reference_visits_labeled = []
  for visit in evaluated_reconstructed_tab_focus_times:
    start_time = visit['start']
    url = visit['url']
    domain = url_to_domain(url)
    closest_time = closest_visit_times_for_reference.find_closest_visit(url, start_time)
    visit_labeled = {k:v for k,v in visit.items()}
    if 'transition' in visit:
      transition = visit['transition']
      reconstructed_transition_types_total[transition] += 1
    if closest_time == None or abs(start_time - closest_time) > 5*1000: # more than 5 seconds off
      stats['reconstructed_has_no_corresponding_reference'] += 1
      domains_where_reconstructed_has_no_reference[domain] += 1
      urls_where_reconstructed_has_no_reference[url] += 1
      visit_labeled['have_reference'] = False
      if 'transition' in visit:
        transition = visit['transition']
        reconstructed_transition_types_with_no_corresponding_reference[transition] += 1
    else:
      stats['reconstructed_has_corresponding_reference'] += 1
      visit_labeled['have_reference'] = True
    reconstructed_visits_labeled.append(visit_labeled)
  for visit in evaluated_tab_focus_times:
    start_time = visit['start']
    url = visit['url']
    closest_time = closest_visit_times_for_reconstructed.find_closest_visit(url, start_time)
    visit_labeled = {k:v for k,v in visit.items()}
    if closest_time == None or abs(start_time - closest_time) > 5*1000: # more than 5 seconds off
      stats['reference_has_no_corresponding_reconstructed'] += 1
      domains_where_reference_has_no_reconstructed[domain] += 1
      urls_where_reference_has_no_reconstructed[url] += 1
      visit_labeled['have_reconstructed'] = False
    else:
      stats['reference_has_corresponding_reconstructed'] += 1
      visit_labeled['have_reconstructed'] = True
    reference_visits_labeled.append(visit_labeled)
  return {
    'stats': stats,
    'domains_where_reconstructed_has_no_reference': domains_where_reconstructed_has_no_reference,
    'domains_where_reference_has_no_reconstructed': domains_where_reference_has_no_reconstructed,
    'urls_where_reconstructed_has_no_reference': urls_where_reconstructed_has_no_reference,
    'urls_where_reference_has_no_reconstructed': urls_where_reference_has_no_reconstructed,
    'reconstructed_visits_labeled': reconstructed_visits_labeled,
    'reference_visits_labeled': reference_visits_labeled,
    'reconstructed_transition_types_with_no_corresponding_reference': reconstructed_transition_types_with_no_corresponding_reference,
    'reconstructed_transition_types_total': reconstructed_transition_types_total,
  }

In [9]:
user_to_span_alignment_data = {}

for user in list_users():
  data = get_span_alignment_stats_for_user(user)
  if 'stats' not in data:
    continue
  user_to_span_alignment_data[user] = data


In [10]:
overall_span_alignment_stats = Counter()
for user,data in user_to_span_alignment_data.items():
  stats = data['stats']
  for k,v in stats.items():
    overall_span_alignment_stats[k] += v

print_counter(overall_span_alignment_stats)
# 60% of reference have corresponding reconstructed
# 84% of reconstructed have corresponding reference

reconstructed_has_corresponding_reference 669982
reference_has_corresponding_reconstructed 669503
reconstructed_has_no_corresponding_reference 199591
reference_has_no_corresponding_reconstructed 162384


In [11]:
overall_transitions_with_no_reference = Counter()

for user,data in user_to_span_alignment_data.items():
  for k,v in data['reconstructed_transition_types_with_no_corresponding_reference'].items():
    overall_transitions_with_no_reference[k] += v

print_counter(overall_transitions_with_no_reference)

link 153043
form_submit 20470
reload 10565
typed 6226
generated 4462
auto_bookmark 3377
auto_toplevel 1400
keyword 43
keyword_generated 5


In [12]:
overall_transitions_total_in_evaluated_reconstruction = Counter()

for user,data in user_to_span_alignment_data.items():
  for k,v in data['reconstructed_transition_types_total'].items():
    overall_transitions_total_in_evaluated_reconstruction[k] += v

print_counter(overall_transitions_total_in_evaluated_reconstruction)


link 669722
form_submit 108915
reload 31720
typed 22748
auto_bookmark 17769
generated 14964
auto_toplevel 3470
keyword 259
keyword_generated 6


In [13]:
# history transition types that have highest levels of no-corresponding-reference
for k,v in overall_transitions_total_in_evaluated_reconstruction.items():
  bad_transitions = overall_transitions_with_no_reference.get(k, 0)
  print k, (float(bad_transitions)/v)

form_submit 0.18794472754
auto_bookmark 0.190050087231
keyword 0.166023166023
typed 0.273694390716
auto_toplevel 0.403458213256
reload 0.333070617907
generated 0.298182304197
link 0.228517205647
keyword_generated 0.833333333333


In [14]:
overall_transitions_total_in_reconstruction = Counter()

for user in list_users_with_hist():
  for visit in get_reconstruct_focus_times_baseline_for_user(user):
    transition = visit['transition']
    overall_transitions_total_in_reconstruction[transition] += 1

print_counter(overall_transitions_total_in_reconstruction)

link 4996461
form_submit 767306
reload 269928
typed 256299
auto_bookmark 135748
generated 131291
auto_toplevel 99731
keyword 2170
keyword_generated 20


In [15]:
overall_transitions_total = Counter()

for user in list_users_with_hist():
  for visit in get_history_ordered_visits_for_user(user):
    transition = visit['transition']
    overall_transitions_total[transition] += 1

print_counter(overall_transitions_total)

link 6157728
form_submit 1099071
reload 412746
typed 265271
auto_bookmark 140369
manual_subframe 137829
generated 131656
auto_toplevel 100801
keyword 2173
auto_subframe 302
keyword_generated 21


In [16]:
for k,v in overall_transitions_total.items():
  bad_transitions = overall_transitions_with_no_reference.get(k, 0)
  print k, (float(bad_transitions)/v)

form_submit 0.0186248204165
auto_subframe 0.0
auto_bookmark 0.0240580185084
keyword 0.0197883110907
typed 0.0234703378809
auto_toplevel 0.0138887511037
reload 0.0255968561779
generated 0.0338913532236
link 0.0248538097168
manual_subframe 0.0
keyword_generated 0.238095238095


In [17]:
for k,v in overall_transitions_total_in_reconstruction.items():
  bad_transitions = overall_transitions_with_no_reference.get(k, 0)
  print k, (float(bad_transitions)/v)



form_submit 0.0266777530737
auto_bookmark 0.0248769779297
keyword 0.0198156682028
typed 0.0242919402729
auto_toplevel 0.0140377615786
reload 0.0391400669808
generated 0.0339855740302
link 0.0306302801123
keyword_generated 0.25


In [18]:
optimal_reconstruction_stats_spans = Counter()
optimal_reconstruction_stats_times = Counter()

for user,data in user_to_span_alignment_data.items():
  for visit in data['reconstructed_visits_labeled']:
    if visit['have_reference']:
      optimal_reconstruction_stats_spans['reconstructed_has_reference'] += 1
    else:
      optimal_reconstruction_stats_spans['reconstructed_has_no_reference'] += 1
  for visit in data['reference_visits_labeled']:
    length = visit['end'] - visit['start']
    if visit['have_reconstructed']:
      optimal_reconstruction_stats_spans['reference_has_reconstructed'] += 1
      optimal_reconstruction_stats_times['seconds_of_reference_that_can_be_reconstructed'] += length
    else:
      optimal_reconstruction_stats_spans['reference_has_no_reconstructed'] += 1
      optimal_reconstruction_stats_times['seconds_of_reference_that_cannot_be_reconstructed'] += length

In [19]:
print_counter(optimal_reconstruction_stats_spans)
# reference has reconstructed 60% = 80332.0 / (80332 + 54018)
# reconstructed has reference 84% = 55748.0 / (10948 + 55748)

# reference has reconstructed 73% = 574617.0/(574617+207343)
# reconstructed has reference 79% = 572765.0/(572765 + 150460)

reconstructed_has_reference 669982
reference_has_reconstructed 669503
reconstructed_has_no_reference 199591
reference_has_no_reconstructed 162384


In [20]:
print_counter(optimal_reconstruction_stats_times)
# seconds of reference that can be reconstructed 57% = 1254885375.005127 / (1254885375.005127 + 947693694.9926758)

seconds_of_reference_that_can_be_reconstructed 12425713372.0
seconds_of_reference_that_cannot_be_reconstructed 3748728104.0


In [21]:
domain_to_seconds_that_cannot_be_reconstructed = Counter()
url_to_seconds_that_cannot_be_reconstructed = Counter()
domain_to_spans_that_cannot_be_reconstructed = Counter()
url_to_spans_that_cannot_be_reconstructed = Counter()

for user,data in user_to_span_alignment_data.items():
  for visit in data['reference_visits_labeled']:
    length = visit['end'] - visit['start']
    url = visit['url']
    domain = url_to_domain(url)
    if not visit['have_reconstructed']:
      domain_to_seconds_that_cannot_be_reconstructed[domain] += length
      domain_to_spans_that_cannot_be_reconstructed[domain] += 1
      url_to_seconds_that_cannot_be_reconstructed[url] += length
      url_to_spans_that_cannot_be_reconstructed[url] += 1

In [22]:
print_counter(domain_to_spans_that_cannot_be_reconstructed)

www.mturk.com 49812
newtab 26618
www.facebook.com 7653
www.youtube.com 5016
mail.google.com 5003
www.google.com 4764
www.reddit.com 3052
www.netflix.com 1759
hitgrabber.net 1557
www.mturkgrind.com 1302
docs.google.com 1297
apps.facebook.com 993
www.amazon.com 934
www.classmates.com 890
us20.chatzy.com 749
www.twitch.tv 731
mturkforum.com 712
www.trulia.com 669
play.pokemonshowdown.com 627
klbibkeccnjlkjkiokjodocebajanakg 579
www.google.co.in 518
www.swagbucks.com 443
bookmarks 427
www.buzzfeed.com 399
 387
us-mg5.mail.yahoo.com 379
www.tumblr.com 359
game.soul-arena.com 354
boards.4chan.org 342
v2.decipherinc.com 340
web.whatsapp.com 307
www.surveyjunkie.com 301
www.surveymonkey.com 298
turkernation.com 298
www.mturkcrowd.com 297
www.hulu.com 296
www.tohosting.tk 291
ma.brazzers.com 291
play.google.com 285
settings 282
www.lipstickalley.com 267
www.host2inger.com 265
www.istreamitall.com 261
twitter.com 259
www.similarweb.com 259
allmyvideos.net 246
www.porn.com 245
www.linkedin.com 23

In [23]:
print_counter(domain_to_seconds_that_cannot_be_reconstructed)

www.mturk.com 828441266.993
www.facebook.com 289283485.997
newtab 232157716.01
www.youtube.com 196993591.002
mail.google.com 117903085.0
www.netflix.com 114632094.002
www.reddit.com 97440090.9995
apps.facebook.com 49416153.9976
www.google.com 43381978.0027
www.mturkgrind.com 37318936.9976
docs.google.com 35249565.0022
hitgrabber.net 34587563.0015
www.amazon.com 30186048.9998
play.pokemonshowdown.com 26162301.9998
www.twitch.tv 24438958.9968
us20.chatzy.com 24294560.0022
mturkforum.com 22611133.0007
www.messenger.com 19367432.9988
www.surveymonkey.com 15802051.0032
boards.4chan.org 15673824.9976
turkernation.com 14286315.9978
www.buzzfeed.com 14148288.0017
us-mg5.mail.yahoo.com 13775991.0015
www.lipstickalley.com 13538586.0012
allmyvideos.net 13465046.0017
www.tumblr.com 12925084.0007
web.whatsapp.com 12691765.9993
www.crowdcurio.com 11523645.0005
www.swagbucks.com 10818555.001
game.soul-arena.com 10798456.0012
www.livenewschat.eu 10287916.9988
www.hulu.com 10262940.9995
twitter.com 100

In [24]:
print_counter(url_to_spans_that_cannot_be_reconstructed)

chrome://newtab/ 26616
https://www.mturk.com/mturk/dashboard 4921
https://www.mturk.com/mturk/findhits?match=true?hit_scraper 4276
https://www.facebook.com/ 4148
https://www.google.com/webhp?sourceid=chrome-instant&ion=1&espv=2&ie=UTF-8 2260
https://www.mturk.com/mturk/findhits?match=false&hit_scraper 1845
http://hitgrabber.net/ 1557
https://mail.google.com/mail/u/0/#inbox 1151
https://us20.chatzy.com/11920092594925 749
https://www.mturk.com/mturk/myhits 745
https://www.mturk.com/HM 638
https://www.mturk.com/mturk/findhits?match=true?hit_scraper-dev 430
https://www.facebook.com/messages/jessica.wood.106 421
https://www.mturk.com/mturk/findhits?match=false 404
http://game.soul-arena.com/ 354
https://www.mturk.com/mturk/findhits?match=true 330
https://www.reddit.com/ 317
https://web.whatsapp.com/ 307
https://mail.google.com/mail/u/0/#spam 260
https://www.mturk.com/mturk/submit 241
https://www.mturk.com/mturk/findhits?match=false&hit_scraper# 221
https://surveys.ipsosinteractive.com/surve

In [25]:
print_counter(url_to_seconds_that_cannot_be_reconstructed)

chrome://newtab/ 232083736.01
https://www.facebook.com/ 167814394.0
https://www.mturk.com/mturk/dashboard 147985049.997
https://www.mturk.com/mturk/findhits?match=true?hit_scraper 126401371.995
https://www.mturk.com/mturk/findhits?match=false&hit_scraper 48501503.0042
http://hitgrabber.net/ 34587563.0015
https://mail.google.com/mail/u/0/#inbox 31891672.0012
https://us20.chatzy.com/11920092594925 24294560.0022
https://www.mturk.com/mturk/findhits?match=true?hit_scraper-dev 18951943.9998
https://mail.google.com/mail/u/0/#spam 18405714.9998
https://www.facebook.com/messages/jessica.wood.106 17004800.0022
https://www.netflix.com/watch/70080629?trackId=13589554 15012666.0002
https://www.mturk.com/mturk/myhits 14057454.9993
https://www.messenger.com/t/darick.harvey 13665289.0002
https://web.whatsapp.com/ 12691765.9993
https://www.mturk.com/HM 12032297.0012
https://www.facebook.com/?q= 10919770.9998
http://game.soul-arena.com/ 10798456.0012
https://www.crowdcurio.com/experiments/13/workflow/ 

In [26]:
overall_domains_where_reconstructed_has_no_reference = Counter()
overall_domains_where_reference_has_no_reconstructed = Counter()
overall_urls_where_reconstructed_has_no_reference = Counter()
overall_urls_where_reference_has_no_reconstructed = Counter()

for user,data in user_to_span_alignment_data.items():
  for k,v in data['domains_where_reconstructed_has_no_reference'].items():
    overall_domains_where_reconstructed_has_no_reference[k] += v
  for k,v in data['domains_where_reference_has_no_reconstructed'].items():
    overall_domains_where_reference_has_no_reconstructed[k] += v
  for k,v in data['urls_where_reconstructed_has_no_reference'].items():
    overall_urls_where_reconstructed_has_no_reference[k] += v
  for k,v in data['urls_where_reference_has_no_reconstructed'].items():
    overall_urls_where_reference_has_no_reconstructed[k] += v


In [27]:
print_counter(overall_urls_where_reconstructed_has_no_reference)

https://www.mturk.com/mturk/dashboard 4708
https://www.google.com/webhp?sourceid=chrome-instant&ion=1&espv=2&ie=UTF-8 3177
https://www.facebook.com/ 2090
https://www.mturk.com/mturk/myhits 1912
https://www.mturk.com/mturk/previewandaccept?groupId=3SFJXQ9EG58Y3WRLXCW2QCVUYYDXDX 1880
https://www.mturk.com/mturk/checkregistration 1778
https://www.mturk.com/mturk/previewandaccept?groupId=35DGDVUOHKKVOAMD7VZRLQEQ0J7YPA 1216
https://www.mturk.com/mturk/previewandaccept?groupId=3NYGPW9OUZ9VT0WI3H1AZXNS6BEZP1 1212
https://perk.tv/video/6268/perk-app-trailers 1127
https://www.mturk.com/mturk/findhits?match=false 986
https://www.mturk.com/mturk/sortmyhits?searchSpec=HITSearch%23T%231%2325%23-1%23T%23%21Status%210%21rO0ABXQACEFzc2lnbmVk%21%23%21Deadline%211%21%23%21 967
https://www.mturk.com/mturk/previewandaccept?groupId=3PBTVBPQ9UI5R0A3PW7WXNE8W4CLGC 908
https://mail.google.com/mail/u/0/#inbox 894
https://perk.tv/video/248/perk-app-trailers 669
https://www.mturk.com/mturk/findhits?match=true 66

In [28]:
print_counter(overall_urls_where_reference_has_no_reconstructed)

chrome://newtab/ 26616
https://www.mturk.com/mturk/dashboard 4921
https://www.mturk.com/mturk/findhits?match=true?hit_scraper 4276
https://www.facebook.com/ 4148
https://www.google.com/webhp?sourceid=chrome-instant&ion=1&espv=2&ie=UTF-8 2260
https://www.mturk.com/mturk/findhits?match=false&hit_scraper 1845
http://hitgrabber.net/ 1557
https://mail.google.com/mail/u/0/#inbox 1151
https://us20.chatzy.com/11920092594925 749
https://www.mturk.com/mturk/myhits 745
https://www.mturk.com/HM 638
https://www.mturk.com/mturk/findhits?match=true?hit_scraper-dev 430
https://www.facebook.com/messages/jessica.wood.106 421
https://www.mturk.com/mturk/findhits?match=false 404
http://game.soul-arena.com/ 354
https://www.mturk.com/mturk/findhits?match=true 330
https://www.reddit.com/ 317
https://web.whatsapp.com/ 307
https://mail.google.com/mail/u/0/#spam 260
https://www.mturk.com/mturk/submit 241
https://www.mturk.com/mturk/findhits?match=false&hit_scraper# 221
https://surveys.ipsosinteractive.com/surve

In [29]:
print_counter(overall_domains_where_reconstructed_has_no_reference)

www.mturk.com 59797
www.google.com 13009
www.youtube.com 9970
www.facebook.com 6408
www.reddit.com 5271
i.imgur.com 3720
docs.google.com 3254
perk.tv 2879
imgur.com 2814
mail.google.com 2652
www.amazon.com 2069
www.mturkgrind.com 1981
mturkforum.com 1697
www.host2inger.com 1359
earnhoney.com 1289
www.tohosting.tk 1203
turkopticon.ucsd.edu 1039
i.ytimg.com 1013
www.surveymonkey.com 996
mail.aol.com 970
vitals-2.crowdcomputingsystems.com 851
work.crowdsurfwork.com 815
www.tumblr.com 714
local.cs-int-592.com:5550 710
www.netflix.com 704
www.google.co.in 694
turkernation.com 650
www.hulu.com 629
www.ebay.com 563
www.pinterest.com 558
www.bing.com 495
payments.amazon.com 490
www.buzzfeed.com 482
www.twitch.tv 479
twitter.com 464
hw.reddit.com 463
www.yelp.com 447
www.instagram.com 447
personalityfactors.net 402
www.yahoo.com 397
www.lipstickalley.com 391
accounts.google.com 390
en.wikipedia.org 389
www.swagbucks.com 370
thewatchseries.to 362
www.car-insurance-quote-fx.com 356
ma.brazzers.co

In [30]:
print_counter(overall_domains_where_reference_has_no_reconstructed)

www.mturk.com 32890
www.facebook.com 17501
www.amazon.com 6178
turkopticon.ucsd.edu 5653
www.mturkcrowd.com 4554
my.presonus.com 4519
www.google.com 4455
www.mturkgrind.com 4400
www.nextmuni.com 4313
aurafc.guildwork.com 3987
prolific.ac 3938
www.reddit.com 3890
news.google.com 3728
mail.google.com 2884
www.youtube.com 2681
vidzi.tv 2679
www.netflix.com 2606
us-mg5.mail.yahoo.com 1997
www.buzzfeed.com 1863
www.hulu.com 1860
docs.python.org 1708
gate.aon.com 1610
armycivilianservice.usajobs.gov 1529
www.tumblr.com 1399
perk.tv 1353
www.txlottery.org 1351
secure.bankofamerica.com 1337
www.playstationtrophies.org 1187
www.kayak.com 1119
www.startribune.com 1109
klbibkeccnjlkjkiokjodocebajanakg 1104
mintvine.com 1096
kat.cr 1012
68.45.57.129:22362 1008
mail.aol.com 990
www.toonova.com 925
knoxville.craigslist.org 923
qbo.intuit.com 896
www.insightexpress.com 862
stonybrookuniversity.co1.qualtrics.com 860
bulletin.uga.edu 840
www.michigantrafficcrashfacts.org 790
sewg.az1.qualtrics.com 787


In [31]:
user_to_evaluation_stats = {}

for user in list_users():
  try:
    stats = get_evaluation_stats_for_user(user)
  except:
    continue
  user_to_evaluation_stats[user] = stats


In [32]:
overall_evaluation_stats = Counter()
for user,stats in user_to_evaluation_stats.items():
  for k,v in stats.items():
    overall_evaluation_stats[k] += v

print overall_evaluation_stats

Counter({'both_inactive': 168803454, 'correct_url': 11688312, 'ref_inactive_but_rec_active': 2346992, 'ref_active_but_rec_inactive': 2052783, 'incorrect_domain': 1730593, 'correct_domain': 817501})


In [33]:
'''
def get_users_with_tab_focus_data():
  users_with_tab_focus_data = []

  for user in list_users():
    if not sdir_exists('tab_focus_times_for_user/' + user + '.json'):
      continue
    tab_focus_times = get_tab_focus_times_for_user(user)
    ordered_visits = get_history_ordered_visits_for_user(user)
    start_time = tab_focus_times[0]['start']
    ordered_visits_after_start = [x for x in ordered_visits if x['visitTime'] > start_time]
    #print len(tab_focus_times)
    if len(tab_focus_times) > 2000 and len(ordered_visits_after_start) > 400:
      users_with_tab_focus_data.append(user)
  return users_with_tab_focus_data
'''
#print len(users_with_tab_focus_data)

"\ndef get_users_with_tab_focus_data():\n  users_with_tab_focus_data = []\n\n  for user in list_users():\n    if not sdir_exists('tab_focus_times_for_user/' + user + '.json'):\n      continue\n    tab_focus_times = get_tab_focus_times_for_user(user)\n    ordered_visits = get_history_ordered_visits_for_user(user)\n    start_time = tab_focus_times[0]['start']\n    ordered_visits_after_start = [x for x in ordered_visits if x['visitTime'] > start_time]\n    #print len(tab_focus_times)\n    if len(tab_focus_times) > 2000 and len(ordered_visits_after_start) > 400:\n      users_with_tab_focus_data.append(user)\n  return users_with_tab_focus_data\n"

In [34]:
#user = 'dwCCD1uofp' # doing very poorly with this user
#user = 'UrZvGUsI8N'
#user = users_with_tab_focus_data[0]
#print user
#url_to_visits = get_history_visits_for_user(user)

In [35]:
'''
def compute_tab_focus_times_with_events_for_user(user):
  current_session_tracker = SessionTracker(store_events=True)
  for line in get_log_with_mlog_active_times_for_user(user):
    current_session_tracker.process_input(uncompress_data_subfields(line))
  current_session_tracker.end_input()
  return current_session_tracker.get_output()
'''

'\ndef compute_tab_focus_times_with_events_for_user(user):\n  current_session_tracker = SessionTracker(store_events=True)\n  for line in get_log_with_mlog_active_times_for_user(user):\n    current_session_tracker.process_input(uncompress_data_subfields(line))\n  current_session_tracker.end_input()\n  return current_session_tracker.get_output()\n'

In [36]:
#tab_focus_times_with_events = compute_tab_focus_times_with_events_for_user(user)


In [37]:
'''
for line in get_log_with_mlog_active_times_for_user(user):
  data = uncompress_data_subfields(line)
  url = get_focused_tab(data)
  if url != None:
    print url
    print data
    break
'''

'\nfor line in get_log_with_mlog_active_times_for_user(user):\n  data = uncompress_data_subfields(line)\n  url = get_focused_tab(data)\n  if url != None:\n    print url\n    print data\n    break\n'

In [38]:
#print len(tab_focus_times_with_events)
#tab_focus_times_with_events[0]

In [39]:
#tab_focus_times = get_tab_focus_times_for_user(user)
#print len(tab_focus_times)
#print tab_focus_times[0]
#print tab_focus_times

In [40]:
#ordered_visits = get_history_ordered_visits_for_user(user)
#first_site = url_to_visits.keys()[0]
#print url_to_visits[first_site]

In [41]:
'''
ordered_visits_after_start = []
for visit in ordered_visits:
  time = visit['visitTime']
  #if 1458171960840.0 <= time <= 1458172165207.0:
  #  print visit
  if time >= 1458171960840.0:
    ordered_visits_after_start.append(visit)
'''

"\nordered_visits_after_start = []\nfor visit in ordered_visits:\n  time = visit['visitTime']\n  #if 1458171960840.0 <= time <= 1458172165207.0:\n  #  print visit\n  if time >= 1458171960840.0:\n    ordered_visits_after_start.append(visit)\n"

In [42]:
#print len(ordered_visits_after_start)

In [43]:
'''
def to_visit_lengths(ordered_visits):
  focus_time_reconstructor = ReconstructFocusTimesBaseline()
  for visit in ordered_visits:
    focus_time_reconstructor.process_history_line(visit)
  return focus_time_reconstructor.get_output()

reconstructed_visit_lengths = to_visit_lengths(ordered_visits)
'''

'\ndef to_visit_lengths(ordered_visits):\n  focus_time_reconstructor = ReconstructFocusTimesBaseline()\n  for visit in ordered_visits:\n    focus_time_reconstructor.process_history_line(visit)\n  return focus_time_reconstructor.get_output()\n\nreconstructed_visit_lengths = to_visit_lengths(ordered_visits)\n'

In [44]:
#print len(reconstructed_visit_lengths)

In [45]:
#print ref_start_time

In [46]:
#print evaluated_tab_focus_times[0]

In [47]:
'''
for x in reconstructed_visit_lengths:
  url = x['url']
  if 'tmi.stanford.edu' in url:
    print x
    break
'''

"\nfor x in reconstructed_visit_lengths:\n  url = x['url']\n  if 'tmi.stanford.edu' in url:\n    print x\n    break\n"

In [48]:
#print len(evaluated_reconstructed_visits)
#print len(evaluated_tab_focus_times)

In [49]:
#def 

In [50]:
#print evaluated_reconstructed_visits[0]
#print evaluated_tab_focus_times[2]
#print evaluated_reconstructed_visits[-1]
#print evaluated_tab_focus_times[-1]

In [51]:
#print len(result_reference.get_output())
#print len(result_reconstructed.get_output())

In [52]:
'''
for ref_answer,rec_answer in zip(result_reference.get_output(), result_reconstructed.get_output())[:1000]:
  ref_domain = 'None'
  rec_domain = 'None'
  if ref_answer != 0:
    ref_url = result_reference.get_url_for_index(ref_answer)
    ref_domain = url_to_domain(ref_url)
  if rec_answer != 0:
    rec_url = result_reconstructed.get_url_for_index(rec_answer)
    rec_domain = url_to_domain(rec_url)
  print ref_domain, rec_domain
'''

"\nfor ref_answer,rec_answer in zip(result_reference.get_output(), result_reconstructed.get_output())[:1000]:\n  ref_domain = 'None'\n  rec_domain = 'None'\n  if ref_answer != 0:\n    ref_url = result_reference.get_url_for_index(ref_answer)\n    ref_domain = url_to_domain(ref_url)\n  if rec_answer != 0:\n    rec_url = result_reconstructed.get_url_for_index(rec_answer)\n    rec_domain = url_to_domain(rec_url)\n  print ref_domain, rec_domain\n"

In [53]:
#for x in evaluated_reconstructed_visits:
#  print x

In [54]:
#for x in evaluated_tab_focus_times:
#  print x

In [55]:
#for x in reconstructed_visit_lengths:
#  url = x['url']
#  if url == 'https://www.mturk.com/mturk/continue?hitId=3ZY8KE4ISJ42CAV8NVJ18F1PI9SQVJ':
#    print x

In [56]:
#for x in tab_focus_times:
#  url = x['url']
#  if url == 'https://www.mturk.com/mturk/continue?hitId=3ZY8KE4ISJ42CAV8NVJ18F1PI9SQVJ':
#    print x

In [57]:

'''
def to_visit_lengths(ordered_visits):
  output = []
  for item,idx in ordered_visits:
    url = item['url']
    visit_time = item['visitTime'] # timestamp in milliseconds
    result = {'url': url, 'start': visit_time, 'active': visit_time}
    visit_duration = 30*1000 # 30 seconds in milliseconds
    if idx < len(ordered_visits):
      next_item = ordered_visits[idx+1]
      next_visit_time = next_item['visitTime']
      time_until_next_visit = next_visit_time - visit_time
      if visit_duration > time_until_next_visit: # more than 30 minutes until next visit
        
      visit_duration = min(visit_duration, time_until_next_visit)
    output.append({'url': url, 'start': visit_time, })
'''

"\ndef to_visit_lengths(ordered_visits):\n  output = []\n  for item,idx in ordered_visits:\n    url = item['url']\n    visit_time = item['visitTime'] # timestamp in milliseconds\n    result = {'url': url, 'start': visit_time, 'active': visit_time}\n    visit_duration = 30*1000 # 30 seconds in milliseconds\n    if idx < len(ordered_visits):\n      next_item = ordered_visits[idx+1]\n      next_visit_time = next_item['visitTime']\n      time_until_next_visit = next_visit_time - visit_time\n      if visit_duration > time_until_next_visit: # more than 30 minutes until next visit\n        \n      visit_duration = min(visit_duration, time_until_next_visit)\n    output.append({'url': url, 'start': visit_time, })\n"